In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/Shareddrives/Data Analytics in Cybersecurity Team"

Mounted at /content/drive
 10k-most-common.txt
 10-million-password-list-top-10000.txt
'An Online Password Guessing Method Based on Big Data.pdf'
'A password generator tool to increase users awareness on bad password construction strategies.pdf'
 CSCE489_704_Final_Project_Spring_2021.docx
 data.csv
 output_file.txt
 passlist.csv
 PasswordClassifier.ipynb
'Password Generation and Strength Prediction Presentation.gslides'
 PasswordGenerator.ipynb
'Password Strength Prediction Using Supervised Machine Learning Techniques.pdf'
'Project Checkpoint 1.gdoc'
'Project Checkpoint 2.gdoc'
'Project Proposal.gdoc'
'Research Paper.gdoc'
 top204000passwords.txt
 top4800passwords.txt
 top64passwords.txt


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time
import random

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
path = '/content/drive/Shareddrives/Data Analytics in Cybersecurity Team/top4800passwords.txt'
text = open(path, 'rb').read().decode(encoding='utf-8')
allchars = "qwertyuiopasdfghjklzxcvbnm1234567890-=!@#$%^&*()_+[];',./{}:\"<>?|\\~`"
print(f'Length of text: {len(text)} characters')
allchars = sorted(set(text))


Length of text: 50074 characters


In [ ]:
# Convert characters to number IDs
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(allchars))
# convert number IDs to characters
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)
# Function to convert number IDS into string
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(13):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

p
a
s
s
w
o
r
d



1
2
3


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
print(dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


In [ ]:
# Batch size
BATCH_SIZE = 32

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(allchars)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.gru2 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

(32, 100, 60) # (batch_size, sequence_length, vocab_size)
Input:
 b'era\r\nabundance\r\nNicholas\r\nMichael1\r\n9999999999\r\n1qazzaq1\r\n13243546\r\n12qw34er\r\n123456ab\r\nzaragoza\r\nwo'

Next Char Predictions:
 b'noRjPDfy8hNiRdW2F@lk@c1wgvphSv4kM0ABfvxB5h0r@n\rARVWpFjOj9zd52lIoIkS@IAlpdGqxVDLoiw7wO[UNK]7m;3Ifzgs3aO2'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
15/15 [==============================] - 3s 99ms/step - loss: 3.8979
Epoch 2/30
15/15 [==============================] - 1s 91ms/step - loss: 3.0523
Epoch 3/30
15/15 [==============================] - 1s 89ms/step - loss: 2.6463
Epoch 4/30
15/15 [==============================] - 1s 89ms/step - loss: 2.4634
Epoch 5/30
15/15 [==============================] - 1s 89ms/step - loss: 2.3212
Epoch 6/30
15/15 [==============================] - 1s 90ms/step - loss: 2.2425
Epoch 7/30
15/15 [==============================] - 1s 89ms/step - loss: 2.1930
Epoch 8/30
15/15 [==============================] - 1s 90ms/step - loss: 2.1576
Epoch 9/30
15/15 [==============================] - 1s 89ms/step - loss: 2.1362
Epoch 10/30
15/15 [==============================] - 1s 90ms/step - loss: 2.1145
Epoch 11/30
15/15 [==============================] - 1s 91ms/step - loss: 2.0995
Epoch 12/30
15/15 [==============================] - 1s 90ms/step - loss: 2.0725
Epoch 13/30
15/15 [==================

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['', '[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['pas'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

pass
davinaus
crumborle
chinitar
caslinal
cantroot
barriabax
appardeon
athenlian
a102a324
1a1x2c123
windechon
versenes
vingblose
vematora
opellome
mariatova
kanalova
langagee
levelduc
raisher1
redrelson
ropedest
refforder
repordce
rockert1
milligal
mandusin
dleffosh
clamball
comanchtons
cacrielda
chappine
calovial
caurwbet
cacamone
enovakin
christian
changhas
calorant
conistra
calefers
concrunce
clatemis
chorling
camestan
caccesane
cabriftro
chomatox
bordiald
bisefann
beliermon
bablands
babcoballa
abracides
abriness
astardich
ancharce
asdffra1
wercemen
20020001
windivin
walmcisa
wallo123
vessions
pendoure
playquelen
netperin
muncuious
mardigan
lacklasd
lif4fick
loveline
liferpiol
lingslaph
lairtome
insedrit
hobzybol
halicala
heariume
gaterine
adfidectir
angolena
alivence
alchanger
angelsen
angyendel
arabulla
alucidet
averter1
asdfzosd
wermering
une4rmat
antiancu
Micrizan
nathblah
canauralo
cas 

________________________________________________________________________________

Run time: